In [ ]:
import wheel_loader
wheel_loader.add_wheels()

In [ ]:
from snowflake.snowpark import Session
from snowflake.connector.cursor import SnowflakeCursor
from snowflake.connector.converter import SnowflakeConverter

from dbt.adapters.snowflake.connections import SnowflakeConnectionManager

# we need to setup paramstyle to pyformat
import snowflake.connector
snowflake.connector.paramstyle = "pyformat"

session = Session.builder.getOrCreate()

# disable colorama output notebooks output does not support it
import os
os.environ['CLICOLOR'] = '0'


from snowflake.snowpark._internal.utils import is_in_stored_procedure

# we mock a few methods
def _process_params_dict(self, params, cursor):
    try:
        res = {k: self._process_single_param(v) for k, v in params.items()}
        return res
    except Exception as e:
        raise Exception("Failed processing pyformat-parameters: {e}")
def _process_params_pyformat(self,params, cursor):
        if params is None:
            return {}
        if isinstance(params, dict):
            return self._process_params_dict(params,cursor)
        if not isinstance(params, (tuple, list)):
            params = [params,]
        try:
            res = map(self._process_single_param, params)
            ret = tuple(res)
            return ret
        except Exception as e:
            raise Exception(f"Failed processing pyformat-parameters; {self}{params} {e}")
def _process_single_param(self, param):
        to_snowflake = self.converter.to_snowflake
        escape = self.converter.escape
        _quote = self.converter.quote
        return _quote(escape(to_snowflake(param)))

def patch_for_dbt(session: Session):
    import snowflake.connector.connection
    if is_in_stored_procedure():
        snowflake.connector.connection.SnowflakeConnection = snowflake.connector.connection.StoredProcConnection
        setattr(snowflake.connector.connection.StoredProcConnection,"_process_params_pyformat",_process_params_pyformat)
        setattr(snowflake.connector.connection.StoredProcConnection,"_process_params_dict",_process_params_dict)
        setattr(snowflake.connector.connection.StoredProcConnection,"_process_single_param",_process_single_param)
    existing_snowflake_connection = session._conn._conn
    setattr(existing_snowflake_connection,"_interpolate_empty_sequences",False)
    existing_snowflake_connection._paramstyle = "pyformat"
    # patching the DBT ConnectionManager to just the current connection
    def new_open_method(cls, connection):
        handle=Session.builder.getOrCreate().connection
        connection.handle = handle
        return connection
    SnowflakeConnectionManager.open = new_open_method

patch_for_dbt(session)

import warnings
warnings.filterwarnings("ignore")


In [ ]:
create database if not exists  jaffle_shop;
create schema if not exists  jaffle_shop.mrojas;

In [ ]:
session.use_database("jaffle_shop")
session.use_schema("mrojas")
session.use_warehouse("xsmall_wh")


In [ ]:
import os
os.makedirs(os.path.expanduser("~/.dbt"),exist_ok=True)
with open(os.path.join(os.path.expanduser("~/.dbt"),"profiles.yml"),"w") as f:
    f.write(f"""
jaffle_shop:
 outputs:
   dev:
     account: {session.get_current_account()}
     database: {session.get_current_database()}
     password: xxx
     role: ACCOUNTADMIN
     schema: {session.get_current_schema()}
     threads: 1
     type: snowflake
     user: user
     warehouse: {session.get_current_warehouse()}
 target: dev

""")

In [ ]:
from dbt.cli.main import dbtRunner, dbtRunnerResult
import colorama
colorama.init()
# initialize
dbt = dbtRunner()
# create CLI args as a list of strings
cli_args = ["seed"]
# run the command
res: dbtRunnerResult = dbt.invoke(cli_args)
# inspect the results
for r in res.result:
    print(f"{r.node.name}: {r.status}")

In [ ]:
from dbt.cli.main import dbtRunner, dbtRunnerResult
import colorama
colorama.init()
# initialize
dbt = dbtRunner()
# create CLI args as a list of strings
cli_args = ["run"]
# run the command
res: dbtRunnerResult = dbt.invoke(cli_args)
# inspect the results
for r in res.result:
    print(f"{r.node.name}: {r.status}")

In [ ]:
select * from customers